# Mini projet Qualité de Données : Détections des doublons
## ***Christophe COMPAIN / Sander COHEN***

### Objectif et Données Disponibles
L'objectif du projet est d'identifier les logiciels vendus sur les deux plateformes.
Pour ce faire, nous disposons des données pour chacune des plateformes isolément, respectivement dans les fichiers ***Company1.csv*** et ***Company2.csv***. 

### Import packages, Variables Globales et import csv

In [3]:
import pandas as pd
import nltk
import time
import numpy as np
import pandas as pd
import math
import re
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ccompain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ccompain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
#path = "D:\\OneDrive - Université Paris-Dauphine\\Bureau\\Cours Master\\12-Qualité de Données\\\Projet\\mini-projet\\"
path = "/Users/ccompain/Documents/code/Dauphine/MasterIASD_prepa_data/mini-projet/github/"
#file1= "Data\\Company1.csv" #"SampleData\\Sample_Company1.csv"
file1 = "Data/Company1.csv"
#file2= "Data\\Company2.csv" #"SampleData\\Sample_Company2.csv"
file2 = "Data/Company1.csv"
#real= "Data\\Ground_truth_mappings.csv" #"SampleData\\Sample_Groud_truth_mappings.csv"
real= "Data/Ground_truth_mappings.csv" #"SampleData\\Sample_Groud_truth_mappings.csv"

In [9]:
company1 = pd.read_csv(path+file1, encoding = "ISO-8859-1")
company2 = pd.read_csv(path+file2, encoding = "ISO-8859-1")
ground_truth_matches = pd.read_csv(path+real, encoding = "ISO-8859-1").drop_duplicates()

### Exploration des données

#### Aspects généraux

In [10]:
company1.head(5)

,id,title,description,manufacturer,price
0,b000jz4hqo,clickart 950 000 - premier image pack (dvd-rom),NaN,broderbund,0.00
1,b0006zf55o,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops ...,computer associates,0.00
2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),NaN,victory multimedia,0.00
3,b000g80lqo,peachtree by sage premium accounting for nonpr...,peachtree premium accounting for nonprofits 20...,sage software,599.99
4,b0006se5bq,singing coach unlimited,singing coach unlimited - electronic learning ...,carry-a-tune technologies,99.99


In [24]:
company1.count()

id              1363
title           1363
description     1248
manufacturer    1363
price           1363
dtype: int64

In [11]:
company2.head(5)

,id,title,description,manufacturer,price
0,b000jz4hqo,clickart 950 000 - premier image pack (dvd-rom),NaN,broderbund,0.00
1,b0006zf55o,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops ...,computer associates,0.00
2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),NaN,victory multimedia,0.00
3,b000g80lqo,peachtree by sage premium accounting for nonpr...,peachtree premium accounting for nonprofits 20...,sage software,599.99
4,b0006se5bq,singing coach unlimited,singing coach unlimited - electronic learning ...,carry-a-tune technologies,99.99


In [25]:
company2.count()

id              1363
title           1363
description     1248
manufacturer    1363
price           1363
dtype: int64

In [12]:
ground_truth_matches.head(5)

,idCompany1,idCompany2
0,b000jz4hqo,18441480711193821750
1,b00004tkvy,18441110047404795849
2,b000g80lqo,18441188461196475272
3,b0006se5bq,18428750969726461849
4,b00021xhzw,18430621475529168165


In [26]:
ground_truth_matches.count()

idCompany1    1300
idCompany2    1300
dtype: int64

#### Top manufacturers

In [22]:
company1.groupby('manufacturer')['id'].count().sort_values(ascending=False).head(9)

manufacturer
adobe                   87
encore software         76
topics entertainment    73
encore                  62
microsoft               58
aspyr media             27
apple                   26
fogware publishing      19
intuit                  18
Name: id, dtype: int64

In [23]:
company2.groupby('manufacturer')['id'].count().sort_values(ascending=False).head(9)

manufacturer
adobe                   87
encore software         76
topics entertainment    73
encore                  62
microsoft               58
aspyr media             27
apple                   26
fogware publishing      19
intuit                  18
Name: id, dtype: int64

#### Observation d'un premier duplicat

In [7]:
company1[company1.id == ground_truth_matches.idCompany1[1]]

,id,title,description,manufacturer,price
2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),NaN,victory multimedia,0.0


In [8]:
company2[company2.id == ground_truth_matches.idCompany2[1]]

,id,name,description,manufacturer,price
1881,18441110047404795849,the beginners bible: noah's ark activity cente...,NaN,NaN,9.95


### Récupération d'une base de stop_words pour nettoyage des descriptions

In [31]:
stop_words = set(nltk.corpus.stopwords.words('english'))
stop_words.update(["r","v","software","entertainment","inc","usa"])

### Fonction de tokenization et nettoyage de chaine de caractères
 - suppression des caractères non alpha
 - conversion majuscule vers minuscule
 - remplacement d'abbréviations communes
 - lemmatisation

In [32]:
def prep(texte):
    #suppression des caracteres non alphanumériques + tout en minuscule
    texte = re.sub("[^a-zA-Z0-9_]", " ",str(texte)).lower()
    #remplacement de mots
    texte = texte.replace("professional", "pro").replace(" upg "," upgrade ").replace(" dlx "," deluxe ")
    #tokenization par mot
    tokens = nltk.word_tokenize(texte)
    #supreesion des stopwords
    filtered_tokens = [w for w in tokens if not w in stop_words]
#    # Stemming
#    texte = [nltk.stem.SnowballStemmer('english').stem(w) for w in filtered_tokens]
    # Lemmatization
    texte = [nltk.stem.WordNetLemmatizer().lemmatize(w) for w in filtered_tokens]
    #remise sous forme d'une string
    return " ".join(texte)

### Fonction de normalisation des prix
 - suppression caractères non-alpha
 - conversion en minuscule

In [33]:
def retreatprice(texte):
    #suppression des caracteres non alphanumériques + tout en minuscule
    return float(re.sub("[^0-9.]", " ",str(texte)))

### Pre-Processing des 2 dataframes
 - Renommage des colonnes
 - Imputation des "na"
 - Traitement des prix
 - Construction d'une colonne "full data" contenant le titre et le manufacturer sur laquelle la fonction "prep" est appliquée

In [35]:
company1['Company']="company1"
company1=company1.rename(columns={"title": "name"})
company1['name'] = company1['name'].fillna(' ')
company1['manufacturer'] = company1['manufacturer'].fillna(' ')
company1['description'] = company1['description'].fillna(' ')
company1['price'] = company1['price'].fillna(' ')
company1['price_retreat'] = company1['price'].apply(retreatprice)
company1['full data']=company1['manufacturer'].apply(prep) + ' ' + company1['name'].apply(prep) # + ' ' + company1['description'].apply(prep)

company2['Company']="company2"
company2=company2.rename(columns={"title": "name"})
company2['name'] = company2['name'].fillna(' ')
company2['manufacturer'] = company2['manufacturer'].fillna(' ')
company2['description'] = company2['description'].fillna(' ')
company2['price'] = company2['price'].fillna(' ')
company2['price_retreat'] = company2['price'].apply(retreatprice)
company2['full data']=company2['manufacturer'].apply(prep) + ' ' + company2['name'].apply(prep) # + ' ' + company2['description'].apply(prep)

### Création d'un Dataframe "corpus" contenant les données de company1 et company2

In [37]:
corpus = pd.concat([company1, company2],sort=False,ignore_index=True)
#corpus.reset_index(drop=True)
len(corpus)

2726

In [38]:
corpus.tail()

,id,name,description,manufacturer,price,Company,price_retreat,full data
2721,b000cs3s2c,flash remoting 1 alp ret eng cd 2u,- marketing information: macromedia flash remo...,adobe,3314.09,company2,3314.09,adobe flash remoting 1 alp ret eng cd 2u
2722,b00005bigp,shapes,,school zone,9.99,company2,9.99,school zone shape
2723,b000h1df7w,dragon naturally speaking standard v9,dragon naturallyspeaking 9 (standard edition) ...,nuance communications inc.,99.99,company2,99.99,nuance communication dragon naturally speaking...
2724,b000p9cr66,mediarecover,mediarecover gives you the ability to recover ...,aladdin systems,29.99,company2,29.99,aladdin system mediarecover
2725,b000j588g4,photo explosion 3.0,photo explosion 3.0,nova development,29.99,company2,29.99,nova development photo explosion 3 0


### Recherche des mots "unique" dans le "full data" pour réduire l'espace de recherche

In [43]:
allwords = corpus['full data'].str.split(expand=True).stack().value_counts()
stop_unique = set(allwords[allwords==1].index)

def prep2(texte):
    tokens = nltk.word_tokenize(texte)
    #supreesion des stopwords
    filtered_tokens = [w for w in tokens if not w in stop_unique]
    #remise sous forme d'une string
    return " ".join(filtered_tokens)

In [44]:
company1['full data']=company1['full data'].apply(prep2)
company2['full data']=company2['full data'].apply(prep2)

### Fonction d'ientification des doublons
 - après quelques essais pas forcément concluants utilisants la distance de Jacquard, nous avons décidés d'expérimenter TF-IDF
 - TF : Term-Frequency => Renvoie la fréquence de chaque mot dans le corpus
 - IDF : Inverse-Data-Frequency => Calcule du poids des mots
 - le TF-IDF score sera le produit des 2 précédentes valeurs

In [51]:
#fonction tfidf
def filtre_tfidf(corpus, ngram, max_df,sim_lim, stop):
    global ground_truth_matches,number_of_matches, matches, matches_df
    start = time.process_time()
    new_number_of_matches = 0
    new_matches=[]

    vectorizer = TfidfVectorizer(ngram_range=ngram, max_df=max_df,sublinear_tf=True,stop_words=[stop]) 
    vectors = vectorizer.fit_transform(corpus['full data'])
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()

    for i in range(len(company1_light)):
        #try :  
        price1 = float(company1_light.iloc[i,6]) 
        #except : 
        #    price1 = 0
        #tokens1name = nltk.word_tokenize(company1_light.iloc[i,7])
        for j in range(len(company2_light)):
            #try :  
            price2 = float(company2_light.iloc[j,6]) 
            #except : 
            #    price2 = 0
            #tokens2name = nltk.word_tokenize(company2_light.iloc[j,7])
            #ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))
            #jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)
            if price1* price2 == 0 or max(price1, price2)/min(price1, price2)<2:
                try :
                    similarity = np.dot(dense[i],np.transpose(dense[len(company1_light)+j])).item(0)/math.sqrt(np.dot(dense[i],np.transpose(dense[i])).item(0) * np.dot(dense[len(company1_light)+j],np.transpose(dense[len(company1_light)+j])).item(0))
                except : 
                    similarity = 0
                if  similarity > sim_lim: #or jd_ng1_ng2_name<0.2 :# or name_score<=1) :
                    new_number_of_matches = new_number_of_matches +1
                    new_matches.append((company1_light.iloc[i,0],company2_light.iloc[j,0]))

    if new_number_of_matches>0:
        print("New matches: {}".format(new_number_of_matches))
        number_of_matches= number_of_matches + new_number_of_matches
        print("Total matches: {}".format(number_of_matches))
        if matches== []:
            matches = new_matches 
            matches_df = pd.DataFrame(matches)
            matches_df.columns= ['idCompany1','idCompany2']
        else:
            new_matches_df = pd.DataFrame(new_matches)
            new_matches_df.columns= ['idCompany1','idCompany2']
            matches_df = pd.concat([matches_df, new_matches_df],sort=False,ignore_index=True).drop_duplicates()

        diff_df = pd.merge(ground_truth_matches, matches_df, how='outer', indicator='Exist')
        true_positives = diff_df[diff_df.Exist=='both']
        false_positives = diff_df[diff_df.Exist=='right_only']
        false_negatives = diff_df[diff_df.Exist=='left_only']
        print("Number of true positives: {}".format(len(true_positives)))
        print("Number of false positives: {}".format(len(false_positives)))
        print("Number of false negatives: {}".format(len(false_negatives)))
        precision = len(true_positives)/(len(true_positives)+ len(false_positives))
        print("Precision: {}".format(precision))
        recall = len(true_positives)/(len(true_positives)+ len(false_negatives))
        print("Recall: {}".format(recall))
        try :
            f_measure = 2*(precision*recall)/(precision+recall)
            print("F measure: {}".format(f_measure))
        except:
            print("F measure not calculable")
    else:
        print("No new match")
    end = time.process_time()
    print("Processing time: {}".format(end - start))

In [52]:
company1_light=company1
company2_light=company2  

In [53]:
#boucle sur les big manufacturer
big_manufacturer_list=["microsoft","apple","encore","adobe","punch","topic","aspyr","sony","webroot"]
number_of_matches = 0
matches=[]
matches_df=[]
for filtre in big_manufacturer_list :
    for max_ngram in [2,1]:
        try :
            company1_light=company1[~company1.id.isin(matches_df.idCompany1)]
            company2_light=company2[~company2.id.isin(matches_df.idCompany2)]    
        except : 
            company1_light=company1
            company2_light=company2        
        company1_light = company1_light[company1_light['full data'].str.contains(filtre)].reset_index(drop=True)
        company2_light = company2_light[company2_light['full data'].str.contains(filtre)].reset_index(drop=True)
        corpus = pd.concat([company1_light, company2_light],sort=False,ignore_index=True)
        print("{} with ngram=(1,{})".format(filtre,max_ngram))
        filtre_tfidf(corpus, (1,max_ngram), 0.1 ,0.6, filtre)
        print(" ")

microsoft with ngram=(1,2)
New matches: 108
Total matches: 108


ValueError: You are trying to merge on uint64 and object columns. If you wish to proceed you should use pd.concat

In [ ]:
##boucle sur les autres noms
for max_ngram in [3,2,1]:
    company1_light=company1[~company1.id.isin(matches_df.idCompany1)]
    company2_light=company2[~company2.id.isin(matches_df.idCompany2)]    
    corpus = pd.concat([company1_light, company2_light],sort=False,ignore_index=True)
    print("Other entries with ngram=(1,{})".format(max_ngram))
    filtre_tfidf(corpus, (1,max_ngram), 0.01 ,0.5, "")
    print(" ")

In [ ]:
base_false_negatives =false_negatives.merge(corpus.loc[corpus['Company'] == 'company1']
                                            .drop(['Company','name','manufacturer'], inplace=False, axis=1)
                                            .rename(columns = {'id': 'idCompany1','description': 'descr1',
                                                               'price': 'price1','full data': 'full data1'}
                                                    , inplace = False)
                                            , how='inner', on='idCompany1').merge(corpus.loc[corpus['Company'] == 'company2']
                                                                                  .drop(['Company','name','manufacturer'], inplace=False, axis=1)
                                                                                  .rename(columns = {'id': 'idCompany2', 
                                                                                                     'description': 'descr2', 
                                                                                                     'price': 'price2', 
                                                                                                     'full data': 'full data2'}, inplace = False)
                                                                                  , how='inner', on='idCompany2')

In [ ]:
base_false_positives =false_positives.merge(corpus.loc[corpus['Company'] == 'company1']
                                            .drop(['Company','name','manufacturer'], inplace=False, axis=1)
                                            .rename(columns = {'id': 'idCompany1','description': 'descr1',
                                                               'price': 'price1','full data': 'full data1'}
                                                    , inplace = False)
                                            , how='inner', on='idCompany1').merge(corpus.loc[corpus['Company'] == 'company2']
                                                                                  .drop(['Company','name','manufacturer'], inplace=False, axis=1)
                                                                                  .rename(columns = {'id': 'idCompany2', 
                                                                                                     'description': 'descr2', 
                                                                                                     'price': 'price2', 
                                                                                                     'full data': 'full data2'}, inplace = False)
                                                                                  , how='inner', on='idCompany2')

In [ ]:
base_false_positives


In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option("max_rows", None)
base_false_negatives